# Billboard Artists

### Summary

Analysis of Billboard Hot 100 songs based on primary or supporting artist status. Web-scraped weekly hot 100 song lists from Billboard.com, dating back to earliest available data 1958-08-09, then parsed out song titles and artists. First listed artist became main artist, and all additional artists were one-hot-encoded for later analysis. Parsing of artists could use additional rework, to account for songs with multiple main artists.

In [1]:
## URL creation for earliest 'Hot 100' list
from datetime import datetime, date

url_base = 'https://www.billboard.com/charts/hot-100/'
start_date = date(year = 1958, month = 8, day = 9)
url = url_base + str(start_date)
print(url)

https://www.billboard.com/charts/hot-100/1958-08-09


Date range available at time of parsing was from 1958-08-09 to 2020-06-27, making the total number of scraped weeks 3230.

In [2]:
## rankings and other song data pull
!pip install beautifulsoup4
!pip install pandas
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# bs4 html request
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
text = str(soup.find('body'))

# useful regex queries
str_li = '<li class="chart-list__element display--flex">[\s\S]+?</li>'
re_rank = re.compile(r'<span class="chart-element__rank__number">(.+?)</span>')
re_song = re.compile(r'<span class="chart-element__information__song text--truncate color--primary">(.+?)</span>')
re_artist = re.compile(r'<span class="chart-element__information__artist text--truncate color--secondary">(.+?)</span>')
re_delta = re.compile(r'<span class="chart-element__information__delta__text text--default">(.+?)</span>')
re_previous = re.compile(r'<span class="chart-element__information__delta__text text--last">(.+?) Last Week</span>')
re_peak = re.compile(r'<span class="chart-element__information__delta__text text--peak">(.+?) Peak Rank</span>')
re_duration = re.compile(r'<span class="chart-element__information__delta__text text--week">(.+?) Weeks on Chart</span>')
df_weekly = pd.DataFrame(columns = ['Week', 'Rank', 'Song', 'Artist', 'Delta', 'Previous', 'Peak', 'Duration'])

# song data parse
for line in re.findall(str_li, text):
    week = start_date
    rank = re_rank.search(line).groups()[0]
    song = re_song.search(line).groups()[0]
    artist = re_artist.search(line).groups()[0]
    delta = re_delta.search(line).groups()[0]
    previous = re_previous.search(line).groups()[0]
    peak = re_peak.search(line).groups()[0]
    duration = re_duration.search(line).groups()[0]
    new_row = {'Week': str(week), 'Rank': rank, 'Song': song, 'Artist': artist, 'Delta': delta, 'Previous': previous, 'Peak': peak, 'Duration': duration}
    df_weekly = df_weekly.append(new_row, ignore_index = True)
    
df_weekly.head(10)

,Week,Rank,Song,Artist,Delta,Previous,Peak,Duration
0,1958-08-09,1,Poor Little Fool,Ricky Nelson,-,1,1,2
1,1958-08-09,2,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,+52,54,2,2
2,1958-08-09,3,Patricia,Perez Prado And His Orchestra,-1,2,2,2
3,1958-08-09,4,Splish Splash,Bobby Darin,-1,3,3,2
4,1958-08-09,5,When,Kalin Twins,-,5,5,2
5,1958-08-09,6,My True Love,Jack Scott,+2,8,6,2
6,1958-08-09,7,Hard Headed Woman,Elvis Presley With The Jordanaires,-3,4,4,2
7,1958-08-09,8,Rebel-'rouser,Duane Eddy His Twangy Guitar And The Rebels,-2,6,6,2
8,1958-08-09,9,Just A Dream,Jimmy Clanton And His Rockets,+3,12,9,2
9,1958-08-09,9,Willie And The Hand Jive,The Johnny Otis Show,-,9,9,2


Above cell's output shows first ten rows of scraped data, which make up the top ten songs from week 1958-08-09. Additional explanation for rightmost four cells: 'Delta' represents change of song from previous week, 'Previous' depicts previous week's rank, 'Peak' signifies highest rank achieved to date, and 'Duration' signifies weeks spent at peak rank.

**CELL BELOW IS VERY SLOW TO RUN.**

By default, it will not run when this notebook is executed. To manually run it, you will need to convert cell type to 'code' (from 'raw').

In [3]:
## df_weekly import from initial run
!pip install xlrd

df_weekly = pd.read_excel('df_weekly.xlsx', index_col = 0)
# reordering columns and adding unique ID
columns = ['ID'] + df_weekly.columns.to_list()
df_weekly['ID'] = df_weekly['Song'].astype(str) + df_weekly['Artist'].astype(str)
df_weekly = df_weekly[columns]
df_weekly.head()

,ID,Week,Rank,Song,Artist,Delta,Previous,Peak,Duration
0,Poor Little FoolRicky Nelson,1958-08-09,1,Poor Little Fool,Ricky Nelson,-,1,1,2
1,Nel Blu Dipinto Di Blu (Volaré)Domenico Modugno,1958-08-09,2,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,+52,54,2,2
2,PatriciaPerez Prado And His Orchestra,1958-08-09,3,Patricia,Perez Prado And His Orchestra,-1,2,2,2
3,Splish SplashBobby Darin,1958-08-09,4,Splish Splash,Bobby Darin,-1,3,3,2
4,WhenKalin Twins,1958-08-09,5,When,Kalin Twins,-,5,5,2


In [4]:
## pull peak and duration values per unique ID
df_unique = df_weekly[['ID', 'Song', 'Artist']].drop_duplicates().reset_index(drop = True)
for i in range(df_unique.shape[0]):
    df_temp = df_weekly[df_weekly['ID'] == df_unique['ID'][i]]
    df_unique.loc[i, 'Peak'] = df_temp['Peak'].min()
    df_unique.loc[i, 'Duration'] = df_temp['Duration'].max()
    df_temp['Delta'][df_temp['Delta'] == '-'] = '0'
    df_temp = df_temp['Delta'].str.replace('+', '')
    df_temp = df_temp.astype(float)
    df_unique.loc[i, 'Delta_Up'] = df_temp.max()
    df_unique.loc[i, 'Delta_Down'] = df_temp.min()

df_unique.head()

<ipython-input-4-b5e3662ca238>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Delta'][df_temp['Delta'] == '-'] = '0'
C:\tools\Anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,ID,Song,Artist,Peak,Duration,Delta_Up,Delta_Down
0,Poor Little FoolRicky Nelson,Poor Little Fool,Ricky Nelson,1.0,11.0,1.0,-24.0
1,Nel Blu Dipinto Di Blu (Volaré)Domenico Modugno,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1.0,16.0,52.0,-39.0
2,PatriciaPerez Prado And His Orchestra,Patricia,Perez Prado And His Orchestra,2.0,14.0,5.0,-48.0
3,Splish SplashBobby Darin,Splish Splash,Bobby Darin,3.0,8.0,-1.0,-27.0
4,WhenKalin Twins,When,Kalin Twins,5.0,9.0,2.0,-24.0


The above takes the df_weekly and condenses it into a dataframe of unique song titles and their pertinent rank attributes (rightmost four columns). 

**CELL BELOW IS VERY SLOW TO RUN.**

By default, it will not run when this notebook is executed. To manually run it, you will need to convert cell type to 'code' (from 'raw').

In [5]:
## df_unique import from initial run
df_unique = pd.read_csv('df_unique.txt', index_col = 0)
df_unique.head()

,ID,Song,Artist,Peak,Duration,main,the jordanaires,the rhythm orchids,the belmonts,arnie,...,sosa geek,bugsey,headie one,polo g,emilee,tierra whack,devin dawson,ozuna + j.rey soul,rowdy rebel,halle
0,Poor Little FoolRicky Nelson,Poor Little Fool,Ricky Nelson,1.0,11.0,ricky nelson,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Nel Blu Dipinto Di Blu (Volaré)Domenico Modugno,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1.0,16.0,domenico modugno,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PatriciaPerez Prado And His Orchestra,Patricia,Perez Prado And His Orchestra,2.0,14.0,perez prado and his orchestra,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Splish SplashBobby Darin,Splish Splash,Bobby Darin,3.0,8.0,bobby darin,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,WhenKalin Twins,When,Kalin Twins,5.0,9.0,kalin twins,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
## value_counts for all 'main' artists
df_unique['main'].value_counts()[0:10]

glee cast          206
drake              151
elvis presley      108
taylor swift        92
aretha franklin     71
eminem              70
lil wayne           70
the beatles         69
future              66
kanye west          62
Name: main, dtype: int64

In [8]:
## overview of all artists in supporting columns
df_supporters = df_unique.loc[:, 'the jordanaires':]
df_supporters

,the jordanaires,the rhythm orchids,the belmonts,arnie,johnny,friend,his orch.,the dons,the bluenotes,the moonglows,...,sosa geek,bugsey,headie one,polo g,emilee,tierra whack,devin dawson,ozuna + j.rey soul,rowdy rebel,halle
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
28799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
## save df_artistcount and df_supportercount to excel files
df_artistcount = df_unique['main'].value_counts()
df_artistcount.to_excel('df_artistcount.xlsx')

df_supportercount = df_supporters.sum(axis = 0).sort_values(ascending = False)
df_supportercount.to_excel('df_supportercount.xlsx')